# Chapter 6. 디스크립터

## 개요
### 디스크립터 메커니즘
+ 최소 두 개의 클래스가 필요
+ 디스크립터 프로토콜 중 하나를 구현한 클래스 인트턴스를 클래스 속성으로 포함해야 함
    + \_\_get\_\_
    + \_\_set\_\_
    + \_\_delete\_\_
    + \_\_set_name\_\_

In [1]:
""" 디스크립터는 __get__ 매직 메서드의 결과를 반환함 """

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class DescriptorClass:
    def __get__(self, instance, owner): # owner의 인스턴스와 클래스를 받음
        if instance is None:
            return self
        logger.info(
            "Call: %s.__get__(%r, %r)", 
            self.__class__.__name__,
            instance,
            owner,
        )
        return instance


class ClientClass:
    descriptor = DescriptorClass()

In [2]:
client = ClientClass()
client.descriptor

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x7fc05c0d6e90>, <class '__main__.ClientClass'>)


In [3]:
client.descriptor is client

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x7fc05c0d6e90>, <class '__main__.ClientClass'>)


True

### 디스크립터 프로토콜의 매서드 
#### \_\_get\_\_(self, instance, owner)
+ owner가 있는 경우는 클래스에서 직접 호출하는 경우 instance가 None이기 때문

In [11]:
""" 디스크립터를 인스턴스로 호출할 때와 클래스 속성으로 호출할 때의 차이점 """

class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return self, owner
        logger.info("Call: %s.__get__(%r, %r)",
                    self.__class__.__name__, instance, owner)
        return instance
    
class ClientClass:
    descriptor = DescriptorClass()

In [12]:
ClientClass().descriptor

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x7fc05c08e9d0>, <class '__main__.ClientClass'>)


In [14]:
ClientClass.descriptor # instance가 None이므로 self를 반환

(<__main__.DescriptorClass at 0x7fc05c08ee90>, __main__.ClientClass)

#### \_\_set\_\_(self, instance, value)
+ 디스크립터에 값을 할당할 때 호출되며, \_\_set\_\_( ) 메서드를 구현한 디스크립터에 대해서만 활성화
+ \_\_set\_\_이 구현되지 않은 경우 client.descriptor = "value" 에서 descriptor자체를 덮어쓸 수 있으므로 주의 필요

In [15]:
from typing import Callable, Any # 형 힌트 지원
                                 # Callable[[Arg1Type, Arg2Type], ReturnType] 

class Validation: # 검증함수와 에러 메세지만을 가지고 있고 value는 __set__ 호출 시 전달받음
    """A configurable validation callable."""

    def __init__(
        self, validation_function: Callable[[Any], bool], error_msg: str 
    ) -> None:
        self.validation_function = validation_function
        self.error_msg = error_msg

    def __call__(self, value): # client.descriptor = 42 에서 42
        if not self.validation_function(value):
            raise ValueError(f"{value!r} {self.error_msg}")


class Field:
    """A class attribute with validation functions configured over it."""

    def __init__(self, *validations):
        self._name = None
        self.validations = validations

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]

    def validate(self, value):
        for validation in self.validations:
            validation(value)

    def __set__(self, instance, value): # client.descriptor = 42 처럼 대입 시 실행됨
        self.validate(value) # 여기서 raise ValueError 안된다면 다음 줄에서 대입됨
        instance.__dict__[self._name] = value


class ClientClass:
    descriptor = Field(
        Validation(lambda x: isinstance(x, (int, float)), "is not a number"),
        Validation(lambda x: x >= 0, "is not >= 0"),
    )

In [16]:
client = ClientClass()
client.descriptor = 42
print(client.descriptor)

client.descriptor = -42

42


ValueError: -42 is not >= 0

In [17]:
client.descriptor = "string"

ValueError: 'string' is not a number

#### \_\_delete\_\_(self, instance)
+ name과 email을 None으로 직접 설정하지 못하도록 \_\_set\_\_  함수 구현
+ \_\_ delete\_\_ 를 통해서만 email의 None설정 가능

In [18]:
class ProtectedAttribute:
    def __init__(self, requires_role=None) -> None:
        self.permission_required = requires_role
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __set__(self, user, value):
        if value is None:
            raise ValueError(f"{self._name} can't be set to None")
        user.__dict__[self._name] = value

    def __delete__(self, user):
        if self.permission_required in user.permissions:
            user.__dict__[self._name] = None
        else:
            raise ValueError(
                f"User {user!s} doesn't have {self.permission_required} "
                "permission"
            )


class User:
    """Only users with "admin" privileges can remove their email address."""

    email = ProtectedAttribute(requires_role="admin")

    def __init__(
        self, username: str, email: str, permission_list: list = None
    ) -> None:
        self.username = username
        self.email = email
        self.permissions = permission_list or []

    def __str__(self):
        return self.username

In [20]:
admin = User("root", "root@d.com", ["admin"])
user = User("user", "user@d.com", ["email", "helpdesk"])

In [21]:
del admin.email
admin.email is None

True

In [22]:
del user.email

ValueError: User user doesn't have admin permission

#### \_\_set_name\_\_(self, ower, name)
+ 디스크립터가 처리하려는 속성의 이름을 알려줌
+ 클래스 데코레이터, 메타클래스를 사용해야하는 종전의 방법을 개선함(3.6 부터)

## 디스크립터의 유형
+ \_\_set\_\__, \_\_delete\_\__ 메서드를 구현했다면 데이터 디스크립터로 구분됨
+ \_\_get\_\__만 구현했다면 비데이터 디스크립터로 구분됨

### 비데이터 디스크립터
